In [10]:
import pandas as pd
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '663ff3650c35de4553dbd25b8eb2de0d'
from tmdbv3api import Movie
tmdb_movie = Movie()
import numpy as np

In [4]:
global movie_year
movie_year = 2019

In [5]:
def scrap_wikipedia():
    '''This function scrapes wikipedia for movie data'''
    
    global link
    link = "https://en.wikipedia.org/wiki/List_of_American_films_of_{}".format(movie_year)
    df1 = pd.read_html(link, header=0)[2]
    df2 = pd.read_html(link, header=0)[3]
    df3 = pd.read_html(link, header=0)[4]
    df4 = pd.read_html(link, header=0)[5]
    df = pd.concat([df1, df2, df3, df4], ignore_index=True)
    global data
    data = df

In [6]:
scrap_wikipedia()

In [7]:
data

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,4,Escape Room,Columbia Pictures / Original Film,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,J A N U A R Y,4,Rust Creek,IFC Films,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,J A N U A R Y,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]
3,J A N U A R Y,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5]
4,J A N U A R Y,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6]
...,...,...,...,...,...,...
240,D E C E M B E R,25,Spies in Disguise,20th Century Fox / Blue Sky Studios / Chernin ...,"Nick Bruno, Troy Quane (directors); Brad Copel...",[132]
241,D E C E M B E R,25,Little Women,Columbia Pictures / Regency Enterprises,Greta Gerwig (director/screenplay); Saoirse Ro...,[224]
242,D E C E M B E R,25,1917,Universal Pictures / DreamWorks Pictures,Sam Mendes (director/screenplay); Krysty Wilso...,[225]
243,D E C E M B E R,25,Just Mercy,Warner Bros. Pictures / Participant Media,"Destin Daniel Cretton (director/screenplay), A...",[226]


In [11]:
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

In [12]:
def make_genres_column():
    data['genres'] = data['Title'].map(lambda x: get_genre(str(x)))

In [14]:
make_genres_column()

In [15]:
data['genres']

0                       Horror Thriller Mystery
1                   Thriller Drama Action Crime
2                                      Thriller
3                        Drama Adventure Family
4                                  Comedy Drama
                         ...                   
240    Animation Action Adventure Comedy Family
241                       Drama Romance History
242                    War Drama Action History
243                         Drama Crime History
244                                       Drama
Name: genres, Length: 245, dtype: object

In [16]:
def extract_features():
    global movies_2019
    movies_2019 = data[['Title','Cast and crew','genres']]

In [18]:
extract_features()

In [19]:
movies_2019

,Title,Cast and crew,genres
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Horror Thriller Mystery
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama Action Crime
2,American Hangman,Wilson Coneybeare (director/screenplay); Donal...,Thriller
3,A Dog's Way Home,Charles Martin Smith (director); W. Bruce Came...,Drama Adventure Family
4,The Upside,Neil Burger (director); Jon Hartmere (screenpl...,Comedy Drama
...,...,...,...
240,Spies in Disguise,"Nick Bruno, Troy Quane (directors); Brad Copel...",Animation Action Adventure Comedy Family
241,Little Women,Greta Gerwig (director/screenplay); Saoirse Ro...,Drama Romance History
242,1917,Sam Mendes (director/screenplay); Krysty Wilso...,War Drama Action History
243,Just Mercy,"Destin Daniel Cretton (director/screenplay), A...",Drama Crime History


In [20]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [21]:
def make_directors_column():
    movies_2019['director_name'] = movies_2019['Cast and crew'].map(lambda x: get_director(x))

In [22]:
make_directors_column()

C:\Users\alpho\AppData\Local\Temp\ipykernel_3228\2335132408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_2019['director_name'] = movies_2019['Cast and crew'].map(lambda x: get_director(x))


In [23]:
movies_2019['director_name']

0                Adam Robitel
1                 Jen McGowan
2           Wilson Coneybeare
3        Charles Martin Smith
4                 Neil Burger
                ...          
240    Nick Bruno, Troy Quane
241              Greta Gerwig
242                Sam Mendes
243     Destin Daniel Cretton
244           Chinonye Chukwu
Name: director_name, Length: 245, dtype: object

In [24]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [40]:
def make_actor_1_column():
    movies_2019['actor_1_name'] = movies_2019['Cast and crew'].map(lambda x: get_actor1(x))
    

In [41]:
make_actor_1_column()

C:\Users\alpho\AppData\Local\Temp\ipykernel_3228\687892879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_2019['actor_1_name'] = movies_2019['Cast and crew'].map(lambda x: get_actor1(x))


In [43]:
movies_2019['actor_1_name']

0           Taylor Russell
1        Hermione Corfield
2        Donald Sutherland
3      Bryce Dallas Howard
4           Bryan Cranston
              ...         
240             Will Smith
241          Saoirse Ronan
242          George MacKay
243      Michael B. Jordan
244          Alfre Woodard
Name: actor_1_name, Length: 245, dtype: object

In [44]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [45]:
def make_actor_2_column():
    movies_2019['actor_2_name'] = movies_2019['Cast and crew'].map(lambda x: get_actor2(x))

In [46]:
make_actor_2_column()

C:\Users\alpho\AppData\Local\Temp\ipykernel_3228\2908149210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_2019['actor_2_name'] = movies_2019['Cast and crew'].map(lambda x: get_actor2(x))


In [47]:
movies_2019['actor_2_name']

0              Logan Miller
1               Jay Paulson
2        Vincent Kartheiser
3        Edward James Olmos
4                Kevin Hart
               ...         
240             Tom Holland
241             Emma Watson
242    Dean-Charles Chapman
243              Jamie Foxx
244          Wendell Pierce
Name: actor_2_name, Length: 245, dtype: object

In [48]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [49]:
def make_actor_3_column():
    movies_2019['actor_3_name'] = movies_2019['Cast and crew'].map(lambda x: get_actor3(x))

In [50]:
make_actor_2_column()

In [51]:
movies_2019['actor_3_name']

0      Deborah Ann Woll
1          Sean O'Bryan
2         Oliver Dennis
3       Alexandra Shipp
4         Nicole Kidman
             ...       
240       Rashida Jones
241       Florence Pugh
242         Mark Strong
243         Brie Larson
244         Aldis Hodge
Name: actor_3_name, Length: 245, dtype: object

In [52]:
movies_2019

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Escape Room,"Adam Robitel (director); Bragi F. Schut, Maria...",Horror Thriller Mystery,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll
1,Rust Creek,Jen McGowan (director); Julie Lipson (screenpl...,Thriller Drama Action Crime,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan
2,American Hangman,Wilson Coneybeare (director/screenplay); Donal...,Thriller,Wilson Coneybeare,Donald Sutherland,Vincent Kartheiser,Oliver Dennis
3,A Dog's Way Home,Charles Martin Smith (director); W. Bruce Came...,Drama Adventure Family,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp
4,The Upside,Neil Burger (director); Jon Hartmere (screenpl...,Comedy Drama,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman
...,...,...,...,...,...,...,...
240,Spies in Disguise,"Nick Bruno, Troy Quane (directors); Brad Copel...",Animation Action Adventure Comedy Family,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones
241,Little Women,Greta Gerwig (director/screenplay); Saoirse Ro...,Drama Romance History,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh
242,1917,Sam Mendes (director/screenplay); Krysty Wilso...,War Drama Action History,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong
243,Just Mercy,"Destin Daniel Cretton (director/screenplay), A...",Drama Crime History,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson


In [63]:
def rename_title():
    global movies_2019
    movies_2019 = movies_2019.rename(columns={'Title':'movie_title'})

In [64]:
rename_title()

In [65]:
movies_2019['movie_title']

0            Escape Room
1             Rust Creek
2       American Hangman
3       A Dog's Way Home
4             The Upside
             ...        
240    Spies in Disguise
241         Little Women
242                 1917
243           Just Mercy
244             Clemency
Name: movie_title, Length: 245, dtype: object

In [66]:
def fill_missing_values():
    movies_2019['actor_2_name'] = movies_2019['actor_2_name'].replace(np.nan, 'unknown')
    movies_2019['actor_3_name'] = movies_2019['actor_3_name'].replace(np.nan, 'unknown')

In [67]:
fill_missing_values()

In [68]:
def make_titles_lower_case():
    movies_2019['movie_title'] = movies_2019['movie_title'].str.lower()

In [69]:
make_titles_lower_case()

In [70]:
movies_2019['movie_title']

0            escape room
1             rust creek
2       american hangman
3       a dog's way home
4             the upside
             ...        
240    spies in disguise
241         little women
242                 1917
243           just mercy
244             clemency
Name: movie_title, Length: 245, dtype: object

In [71]:
def combine_features_in_one_column():
    movies_2019['comb'] = movies_2019['actor_1_name'] + ' ' + movies_2019['actor_2_name'] + ' '+ movies_2019['actor_3_name'] + ' '+ movies_2019['director_name'] +' ' + movies_2019['genres']

In [72]:
combine_features_in_one_column()

In [73]:
movies_2019['comb']

0      Taylor Russell Logan Miller Deborah Ann Woll A...
1      Hermione Corfield Jay Paulson Sean O'Bryan Jen...
2      Donald Sutherland Vincent Kartheiser Oliver De...
3      Bryce Dallas Howard Edward James Olmos Alexand...
4      Bryan Cranston Kevin Hart Nicole Kidman Neil B...
                             ...                        
240    Will Smith Tom Holland Rashida Jones Nick Brun...
241    Saoirse Ronan Emma Watson Florence Pugh Greta ...
242    George MacKay Dean-Charles Chapman Mark Strong...
243    Michael B. Jordan Jamie Foxx Brie Larson Desti...
244    Alfre Woodard Wendell Pierce Aldis Hodge Chino...
Name: comb, Length: 245, dtype: object

In [74]:
movies_2019.isna().sum()

movie_title      0
Cast and crew    0
genres           0
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
comb             0
dtype: int64

In [79]:
def get_features():
    global movies_2019
    movies_2019 = movies_2019.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [80]:
get_features()

In [81]:
movies_2019

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll,Horror Thriller Mystery,escape room
1,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan,Thriller Drama Action Crime,rust creek
2,Wilson Coneybeare,Donald Sutherland,Vincent Kartheiser,Oliver Dennis,Thriller,american hangman
3,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp,Drama Adventure Family,a dog's way home
4,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman,Comedy Drama,the upside
...,...,...,...,...,...,...
240,"Nick Bruno, Troy Quane",Will Smith,Tom Holland,Rashida Jones,Animation Action Adventure Comedy Family,spies in disguise
241,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance History,little women
242,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action History,1917
243,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime History,just mercy


In [82]:
def save_data_to_file():
    movies_2019.to_csv('D:/github-repos/movie-recommendation-system-with-sentiment-analysis-with-nlp/data/02_intermediate/movies_of_2019.csv', index=False)

In [83]:
save_data_to_file()